In [33]:
# enable automatic reloading of the notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Evaluation

In [34]:
import json
from src.utils.metrics import get_pearson_r, get_spearman_r, get_kendall_tau

## STS Benchmark

### Metric comparison

| Metric | Pearson R | Spearman R | Kendall Tau |
| ------ | --------- | ---------- | ------------| 
| MEAN   | 0.788     | 0.781      | 0.602       |
| MAX    | 0.786     | 0.776      | 0.597       |
| EMD    | 0.749     | 0.738      | 0.557       |
| SEQ    | 0.696     | 0.685      | 0.509       |
| CLS    | 0.664     | 0.693      | 0.518       |


# Parameter comparison

| Reg 1 | Reg 2 | Pearson R | Spearman R | Kendall Tau |
| ----- | ----- | --------- | ---------- | ------------| 
| 0.1   | 0.1   | 0.696     | 0.685      | 0.509       |
| 1     | 0.1   | 0.707     | 0.696      | 0.519       |
| 1.1   | 0.1   | 0.706     | 0.695      | 0.519       |
| 10    | 0.1   | -         | -          | -           |


In [74]:
sts_results = "../results/sts/scores.json"
with open(sts_results, "r", encoding="utf8") as f:
    results = json.load(f)

In [75]:
x = [-k["system_score"] for k in results]
y = [k["human_score"] for k in results]

In [76]:
get_pearson_r(x, y)

(0.7067123910602223, 3.605407559544885e-209)

In [77]:
get_spearman_r(x, y)

SpearmanrResult(correlation=0.6954184759403137, pvalue=7.531196032940135e-200)

In [78]:
get_kendall_tau(x, y)

KendalltauResult(correlation=0.5186698727988244, pvalue=3.6445733428029304e-176)

## WMT18 Benchmark

In [8]:
sts_results = "../results/wmt18/scores.cs-en.json"
with open(sts_results, "r", encoding="utf8") as f:
    results = json.load(f)

In [9]:
x = [k["system_score"] for k in results]
y = [k["human_score"] for k in results]

In [10]:
get_pearson_r(x, y)

(-0.23013920481429798, 2.1468810046480817e-104)

In [11]:
get_spearman_r(x, y)

SpearmanrResult(correlation=-0.25196709007077767, pvalue=1.9915913193194048e-125)

In [12]:
get_kendall_tau(x, y)

KendalltauResult(correlation=-0.17291941748006723, pvalue=4.7052573636609185e-126)